In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

# Load Datasets

In [2]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [3]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

# SVM Classifier

In [4]:
def finetune_parameters(X_train, y_train):
    # Define the model
    model = SVC()

    # Define hyperparameters to search
    param_grid = {'C': [1, 10, 100],
                  'kernel': ['linear', 'poly', 'rbf']}

    # Perform randomized search for hyperparameter tuning
    randomized_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=3, cv=10, random_state=42)
    randomized_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = randomized_search.best_params_
    print(best_params)

    return best_params

In [5]:
def svm_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Find best parameters using RandomizedSearch
    best_params = finetune_parameters(X_train, y_train)

    # Train the model with the best hyperparameters
    best_model = SVC(**best_params)
    best_model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(best_model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx


In [6]:
def print_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of SVM Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [7]:
val_acc_final, test_acc_final, confusion_mtrx_final = svm_train(X_final, y_final)
print_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

{'kernel': 'rbf', 'C': 10}
Performance of SVM Classification on Final Dataset:
Cross Validation Accuracy =  0.9934723171565276
Test Accuracy =  0.9947916666666666
Confusion Matrix:
[[0.985 0.015]
 [0.    1.   ]]


## Single Dataset

In [8]:
# Due to RandomSearchCV timing out, we used this to manually test the different hyperparameter combos
def svm_train_single(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Train the model with the best hyperparameters
    best_model = SVC(kernel="rbf", C=100)
    best_model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(best_model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx

In [9]:
val_acc_single, test_acc_single, confusion_mtrx_single = svm_train_single(X_single, y_single)
print_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

Performance of SVM Classification on Single Dataset:
Cross Validation Accuracy =  0.9133587786259543
Test Accuracy =  0.9420289855072463
Confusion Matrix:
[[0.963 0.003 0.003 0.    0.015 0.    0.009 0.003 0.003]
 [0.024 0.94  0.006 0.006 0.006 0.    0.012 0.006 0.   ]
 [0.011 0.005 0.941 0.021 0.    0.    0.005 0.    0.016]
 [0.034 0.009 0.009 0.932 0.009 0.    0.    0.    0.009]
 [0.064 0.    0.    0.    0.926 0.01  0.    0.    0.   ]
 [0.053 0.026 0.026 0.    0.    0.882 0.013 0.    0.   ]
 [0.    0.02  0.    0.01  0.01  0.    0.96  0.    0.   ]
 [0.    0.    0.    0.04  0.02  0.    0.    0.94  0.   ]
 [0.    0.    0.011 0.011 0.    0.    0.    0.022 0.955]]
